In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
#import scikitplot as skplt
import tensorflow as tf
import keras
import seaborn as sns

# Carregando base de dados  pré-processada

In [3]:
trainSet = pd.read_csv("../../Database/emailsTrainSet.csv")
trainSet = trainSet.drop(columns=["Unnamed: 0"])
trainSet = trainSet.dropna()
trainTarget = trainSet["target"].array

trainSet
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11930555445333959433
xla_global_id: -1
]


In [4]:
testSet = pd.read_csv("../../Database/emailsTestSet.csv")
testSet = testSet.drop(columns=["Unnamed: 0"])
testSet = testSet.dropna()
testTarget = testSet["target"].array

testSet

,email,target
0,note personn pleased announce effective datee...,0
1,dear orgg japan team disregard datee notice ...,0
2,save medl correct cation percentt pharmsho sp...,1
3,forwarded personn datee ...,0
4,hi genierc gpee viarga caiils available oin...,1
...,...,...
6664,searching place buy med online recently pl...,1
6665,approval process initiated datee meeting event...,0
6666,credit mess want perfect credit nice easily...,1
6667,tue datee cardinall cardinall dear sir hop...,1


In [5]:
emailsTextTrain = []
for email in trainSet["email"]:
    emailsTextTrain.append(email)

In [6]:
emailsTextTest = []
for email in testSet["email"]:
    emailsTextTest.append(email)

# Representação vetorial Bag of Words

In [7]:
vectorizerTrain = CountVectorizer(max_features=2100)
XTrain = vectorizerTrain.fit_transform(emailsTextTrain)

bagTrain = pd.DataFrame(XTrain.toarray(),columns=vectorizerTrain.get_feature_names())

bagTrain

,aa,ability,able,absolutely,accept,acceptance,accepted,access,according,account,...,xanax,xl,xp,yahoo,year,yes,yield,yo,young,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26668,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
26669,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26670,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26671,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
vectorizerTest = CountVectorizer(max_features=2100)
XTest = vectorizerTest.fit_transform(emailsTextTest)

bagTest = pd.DataFrame(XTest.toarray(),columns=vectorizerTest.get_feature_names())

bagTest

,ability,able,absolutely,abuse,accept,acceptance,accepted,access,according,account,...,www,wysak,xl,xp,yahoo,year,yes,yield,young,zone
0,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6663,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6664,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6665,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6666,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Visualização de dados com TSNE

In [9]:
# model = TSNE(n_components=2, random_state=0)
# #model = PCA(n_components=50, svd_solver='full')
# array_red = model.fit_transform(bag)
#
# df_tsne = pd.DataFrame(array_red)
#
# df_tsne['Target'] = target
# df_tsne_c1 = df_tsne[df_tsne['Target'] == 0]
#
# df_tsne_c2 = df_tsne[df_tsne['Target'] == 1]
#
# plt.scatter(df_tsne_c1[0].array,df_tsne_c1[1].array,marker='o',color='blue')
#
# plt.scatter(df_tsne_c2[0].array,df_tsne_c2[1].array,marker='o',color='red')
#
# plt.title('Dados')
# plt.xlabel('x')
# plt.ylabel('y')
#
# plt.show()

# Validação

In [10]:
X_treino, X_teste, y_treino, y_teste = train_test_split(bagTrain.values, trainTarget, test_size=0.2)



In [11]:
X_treino = np.array(X_treino)

X_treino = X_treino.reshape((X_treino.shape[0],1,X_treino.shape[1]))
print(X_treino.shape)

(21338, 1, 2100)


In [12]:
model = keras.models.Sequential([
########## LSTM
keras.layers.LSTM(50, activation='relu', input_shape=[1, X_treino.shape[2]]),
])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                430200    
                                                                 
Total params: 430,200
Trainable params: 430,200
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [15]:
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4,mode='min'), tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0, restore_best_weights=True)]

history = model.fit(np.array(X_treino), np.array(y_treino), epochs=200,validation_split=0.05,callbacks=callbacks)

Epoch 1/200
634/634 [==============================] - 2s 2ms/step - loss: 0.6608 - accuracy: 0.7019 - val_loss: 0.5395 - val_accuracy: 0.7338 - lr: 0.0100
Epoch 2/200
634/634 [==============================] - 1s 2ms/step - loss: 0.4102 - accuracy: 0.8336 - val_loss: 0.3023 - val_accuracy: 0.8847 - lr: 0.0100
Epoch 3/200
634/634 [==============================] - 1s 2ms/step - loss: 0.3091 - accuracy: 0.8819 - val_loss: 0.4151 - val_accuracy: 0.8126 - lr: 0.0100
Epoch 4/200
634/634 [==============================] - 1s 2ms/step - loss: 0.3107 - accuracy: 0.8701 - val_loss: 0.2715 - val_accuracy: 0.8978 - lr: 0.0100
Epoch 5/200
634/634 [==============================] - 1s 2ms/step - loss: 0.2455 - accuracy: 0.9095 - val_loss: 0.2296 - val_accuracy: 0.9194 - lr: 0.0100
Epoch 6/200
634/634 [==============================] - 1s 2ms/step - loss: 0.2170 - accuracy: 0.9231 - val_loss: 0.2062 - val_accuracy: 0.9335 - lr: 0.0100
Epoch 7/200
634/634 [==============================] - 1s 2ms/st

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.xlabel('epoch')
plt.show()

In [ ]:
model.evaluate(np.array(bagTest.values), np.array(testTarget))

In [ ]:
#categories = ["Ham", "Spam"]
#
#skplt.metrics.plot_confusion_matrix(
#    [categories[i] for i in target], [categories[i] for i in predicoes.tolist()],
#    title="Confusion Matrix",
#    cmap="Purples",
#    hide_zeros=True,
#    figsize=(5,5)
#)
#
#plt.xticks()

In [ ]:
# skplt.metrics.plot_confusion_matrix(
#     [categories[i] for i in target], [categories[i] for i in predicoes.tolist()],
#     normalize=True,
#     title="Confusion Matrix",
#     cmap="Purples",
#     hide_zeros=True,
#     figsize=(5,5)
# )
#
# plt.xticks()